This notebook is adapted from https://github.com/fdalvi/NeuroX/blob/master/examples/End%20to%20End%20Example.ipynb

# Imports

In [60]:
import numpy as np

# Add these lines if you clone from https://github.com/fdalvi/NeuroX
# import sys
# sys.path.append("/path/to/NeuroX")
# In the case of the notebook, the path is the parent directory
import sys
sys.path.append("..")

# Data

In [5]:
!wget http://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz -O - | gunzip | cut -f1,2 -d" " > pos.train.txt 
!wget http://www.cnts.ua.ac.be/conll2000/chunking/test.txt.gz -O - | gunzip | cut -f1,2 -d " " > pos.test.txt

--2022-11-14 08:57:10--  http://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz
Resolving www.cnts.ua.ac.be (www.cnts.ua.ac.be)... 146.175.13.81
Connecting to www.cnts.ua.ac.be (www.cnts.ua.ac.be)|146.175.13.81|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz [following]
--2022-11-14 08:57:10--  https://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz
Connecting to www.cnts.ua.ac.be (www.cnts.ua.ac.be)|146.175.13.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611540 (597K) [application/x-gzip]
Saving to: 'STDOUT'

-                   100%[===================>] 597.21K  1.21MB/s    in 0.5s    

2022-11-14 08:57:11 (1.21 MB/s) - written to stdout [611540/611540]

--2022-11-14 08:57:11--  http://www.cnts.ua.ac.be/conll2000/chunking/test.txt.gz
Resolving www.cnts.ua.ac.be (www.cnts.ua.ac.be)... 146.175.13.81
Connecting to www.cnts.ua.ac.be (www.cnts.ua.ac.be)|

In [6]:
def preprocessing(fin, fout):
    fout_text = open(fout+'.text', 'w', encoding='utf-8')
    fout_tag = open(fout+'.tag', 'w', encoding='utf-8')
    sent_text = []
    sent_tag = []
    with open(fin, 'r', encoding='utf-8') as f:
        for line in f:
            if line != '\n':
                [text, tag] = line.split()
                sent_text.append(text)
                sent_tag.append(tag)
            else:
                sent_text.append('\n')
                sent_tag.append('\n')
                fout_text.write(' '.join(sent_text))
                fout_tag.write(' '.join(sent_tag))
                sent_text = []
                sent_tag = []
        fout_text.close()
        fout_tag.close()
        

In [7]:
preprocessing('pos.train.txt', 'train')
preprocessing('pos.test.txt', 'test')

# Extract Representations

In [8]:
import neurox.data.extraction.transformers_extractor as transformers_extractor

In [11]:
# you can run the following command in terminal to avoid verbose output in notebook
# python -m neurox.data.extraction.transformers_extractor --filter_layers 0 bert-base-uncased train.text train_activation.json

# transformers_extractor.extract_representations('bert-base-uncased',
#     'train.text',
#     'train_activation.json',
#     device = 'cuda',
#     aggregation="average", #last, first
#     filter_layers = [0]
# )

In [10]:
! cut -c1-120 < train_activation.json

{"linex_index": 0, "features": [{"token": "confidence", "layers": [{"index": 0, "values": [0.55443066, 0.57546091, -0.58
{"linex_index": 1, "features": [{"token": "chancellor", "layers": [{"index": 0, "values": [-0.10590585, -1.0280143, -0.6
{"linex_index": 2, "features": [{"token": "but", "layers": [{"index": 0, "values": [0.37906024, 0.5005427, -0.0606893, 0
{"linex_index": 3, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.64851344, 0.67392409, -0.0932498
{"linex_index": 4, "features": [{"token": "``", "layers": [{"index": 0, "values": [-0.73166448, 0.1452198, -0.77601731, 
{"linex_index": 5, "features": [{"token": "``", "layers": [{"index": 0, "values": [-0.73166448, 0.1452198, -0.77601731, 
{"linex_index": 6, "features": [{"token": "forecast##s", "layers": [{"index": 0, "values": [-0.79362893, -0.15296265, 0.
{"linex_index": 7, "features": [{"token": "the", "layers": [{"index": 0, "values": [-0.43669656, 0.53601712, -0.05141335
{"linex_index": 8, "features": [

# Prepare Data

In [9]:
import neurox.data.loader as data_loader
activations, num_layers = data_loader.load_activations('train_activation.json', 768)

Loading json activations from train_activation.json...
8936 1.0


In [13]:
print(len(activations))

8936


In [16]:
activations[0].shape

(37, 768)

In [15]:
activations[1].shape

(27, 768)

In [17]:
# load_data also does sanity checks for parallelism between tokens, labels and activations
tokens = data_loader.load_data('train.text',
                               'train.tag',
                               activations,
                               512 # max_sent_l
                              )

In [18]:
import neurox.interpretation.utils as utils
X, y, mapping = utils.create_tensors(tokens, activations, 'NN')
label2idx, idx2label, src2idx, idx2src = mapping

Number of tokens:  211727
length of source dictionary:  19122
length of target dictionary:  44
211727
Total instances: 211727
['voiced', 'Electrochemical', '91', 'cowboys', 'anti-takeover', 'Bonfire', 'rulers', 'school', 'inducement', 'Administration', 'dissidents', 'friendly', 'indelible', '204.5', '2179.1', '96', 'Mayer', 'Negotiable', 'Ciminero', 'Johan']
Number of samples:  211727
Stats: Labels with their frequencies in the final set
UH 15
NNS 13619
NNPS 420
DT 18335
WDT 955
. 8827
EX 206
PRP$ 1881
# 36
, 10770
MD 2167
'' 1493
WP$ 35
FW 38
TO 5081
NNP 19884
JJ 13085
$ 1750
CC 5372
WP 529
JJS 374
JJR 853
WRB 478
RP 83
( 274
) 281
: 1047
VBP 2868
VBG 3272
PRP 3820
VBN 4763
RB 6607
IN 22764
CD 8315
RBS 191
PDT 55
VB 6017
RBR 321
NN 30147
VBZ 4648
POS 1769
VBD 6745
SYM 6
`` 1531


In [19]:
X.shape

(211727, 768)

In [20]:
print(mapping[0])

{'UH': 0, 'NNS': 1, 'NNPS': 2, 'DT': 3, 'WDT': 4, '.': 5, 'EX': 6, 'PRP$': 7, '#': 8, ',': 9, 'MD': 10, "''": 11, 'WP$': 12, 'FW': 13, 'TO': 14, 'NNP': 15, 'JJ': 16, '$': 17, 'CC': 18, 'WP': 19, 'JJS': 20, 'JJR': 21, 'WRB': 22, 'RP': 23, '(': 24, ')': 25, ':': 26, 'VBP': 27, 'VBG': 28, 'PRP': 29, 'VBN': 30, 'RB': 31, 'IN': 32, 'CD': 33, 'RBS': 34, 'PDT': 35, 'VB': 36, 'RBR': 37, 'NN': 38, 'VBZ': 39, 'POS': 40, 'VBD': 41, 'SYM': 42, '``': 43}


In [22]:
X[0].shape 

(768,)

train_val split

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [36]:
X_train.shape

(158795, 768)

In [37]:
X_train.shape[0]/X_test.shape[0]

2.9999811078364695

# Train Probing Classifier

In [23]:
import neurox.interpretation.linear_probe as linear_probe

In [38]:
probe = linear_probe.train_logistic_regression_probe(X_train, y_train, lambda_l1=0.001, lambda_l2=0.001, num_epochs=20, batch_size=64)

Training classification probe
Creating model...
Number of training instances: 158795
Number of classes: 44


epoch [1/20]: 0it [00:00, ?it/s]

Epoch: [1/20], Loss: 0.0154


epoch [2/20]: 0it [00:00, ?it/s]

Epoch: [2/20], Loss: 0.0132


...

Epoch: [18/20], Loss: 0.0127


epoch [19/20]: 0it [00:00, ?it/s]

Epoch: [19/20], Loss: 0.0127


epoch [20/20]: 0it [00:00, ?it/s]

Epoch: [20/20], Loss: 0.0127


# Evaluate Model

In [39]:
scores = linear_probe.evaluate_probe(probe, X_test, y_test, idx_to_class=idx2label)
scores

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.89


{'__OVERALL__': 0.8854379203506385,
 'UH': 0.0,
 'NNS': 0.9833786642490179,
 'NNPS': 0.0,
 'DT': 0.9783807632914184,
 'WDT': 0.5100401606425703,
 '.': 0.9897229669347631,
 'EX': 0.0,
 'PRP$': 1.0,
 '#': 0.0,
 ',': 1.0,
 'MD': 0.9300699300699301,
 "''": 1.0,
 'WP$': 0.0,
 'FW': 0.0,
 'TO': 1.0,
 'NNP': 0.6624975183641056,
 'JJ': 0.7946428571428571,
 '$': 1.0,
 'CC': 0.9902255639097745,
 'WP': 1.0,
 'JJS': 0.29411764705882354,
 'JJR': 0.7914893617021277,
 'WRB': 0.8867924528301887,
 'RP': 0.0,
 '(': 0.0,
 ')': 0.0,
 ':': 0.936,
 'VBP': 0.6882022471910112,
 'VBG': 0.8265060240963855,
 'PRP': 0.9909182643794148,
 'VBN': 0.8190082644628099,
 'RB': 0.9037656903765691,
 'IN': 0.9977292576419214,
 'CD': 0.9848985370457763,
 'RBS': 0.0,
 'PDT': 0.0,
 'VB': 0.6564337034617896,
 'RBR': 0.0,
 'NN': 0.9004421814283801,
 'VBZ': 0.818105616093881,
 'POS': 0.9356321839080459,
 'VBD': 0.7622504537205081,
 'SYM': 0.0,
 '``': 1.0}

# Get Neuron Ranking

In [42]:
ordering, cutoffs = linear_probe.get_neuron_ordering(probe, label2idx)

  0%|          | 0/101 [00:00<?, ?it/s]

In [43]:
ordering[:10]

[644, 134, 519, 653, 413, 290, 36, 296, 429, 308]

In [44]:
cutoffs

[38,
 44,
 56,
 67,
 83,
 95,
 108,
 123,
 135,
 151,
 168,
 190,
 202,
 218,
 234,
 255,
 271,
 283,
 305,
 319,
 337,
 352,
 370,
 385,
 399,
 417,
 434,
 450,
 465,
 477,
 493,
 516,
 526,
 539,
 556,
 567,
 580,
 589,
 604,
 613,
 621,
 633,
 645,
 653,
 662,
 666,
 672,
 677,
 683,
 689,
 696,
 700,
 709,
 716,
 720,
 723,
 726,
 729,
 736,
 738,
 740,
 742,
 744,
 746,
 750,
 752,
 755,
 756,
 756,
 757,
 758,
 759,
 760,
 761,
 762,
 762,
 764,
 764,
 765,
 765,
 765,
 766,
 767,
 767,
 767,
 767,
 767,
 767,
 767,
 767,
 767,
 767,
 767,
 768,
 768,
 768,
 768,
 768,
 768,
 768,
 768]

# Train on top N neurons

In [74]:
def train_on_neurons(X, y, ordering, percentage, mode, idx2label=None):
    total = X.shape[1]
    N = np.floor(total*percentage).astype(int)
    if mode == 'top':
        keep_neurons = ordering[:N]
    elif mode == 'bottom':
        keep_neurons = ordering[-N:]
    else:
        print('keep should be top or bottom!')
    X_selected = ablation.filter_activations_keep_neurons(X, keep_neurons)
    print(X_selected.shape)
    X_selected_train, X_selected_test, y_selected_train, y_selected_test = train_test_split(X_selected, y)
    probe_selected = linear_probe.train_logistic_regression_probe(X_selected_train, y_selected_train, lambda_l1=0.001, lambda_l2=0.001, batch_size=128)
    scores = linear_probe.evaluate_probe(probe_selected, X_selected_test, y_selected_test, idx_to_class=idx2label)
    return scores

In [75]:
train_on_neurons(X, y, ordering, 0.1, 'top')

(211727, 76)
Training classification probe
Creating model...
Number of training instances: 158795
Number of classes: 44


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.0138


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0098


...

epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0085


Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.78


{'__OVERALL__': 0.7819844328572508}

# Score dropping v.s. 

# Further analysis

In [24]:
top_neurons, top_neurons_per_class = linear_probe.get_top_neurons(probe, 0.01, label2idx)
print(top_neurons)

[  10 8205 6159 8208 8213   21 8219 6184 6186 8237 4149 6201 4163 6212
 4167 4174 6225 4177 4179 2136 2139 4187 8284 8286 6238 6241 2147 4195
 8295 4204 6252  115 4212 8308 4223 6273 8336 8346 8347 6302 4257  168
 8360 6317 2228 2230 8377 6331  189  191 2247  203 8400 6372 8427 2285
 4339 8450 4362 6412 6415 6418 2324 8468 4388 6437  308 4407 6458 8508
 6471 2378  330 6482 8538  355 4456 6508 4464 2421 2423  376  379  382
 6530 2444 6548 8596 6555 4519 6574 4528 6578 6584 4557 2510 8659 8669
 4575 6624 4576  484 6642 2555 6679 8730  541  548 4649  554 6701 2611
 8756 8757 8762 6715 8767 8769 2631 6731  594  599  622 6774 4727 4731
 6786  644 6800 6801 6802  661 2717 8862 2719 2721  678 6823 4777 4781
 2736 2738 4787 6836 8882  694 6838 4791 8891 4798 2751 2768  721 8927
 2785 8931 2791 6887 2793 2794  748  770 4870  775  778 4881 4887  793
 4899 4901 8997  809 2861 4910 4917 4925 4931  849 2897  853 4950  862
 7007  868 2919  873 4972 4973 7021 2929 9073 4978  885 2934 7041 2950
  902 

In [25]:
X_zeroed = ablation.zero_out_activations_keep_neurons(X, top_neurons)
linear_probe.evaluate_probe(probe, X_zeroed, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.96


{'__OVERALL__': 0.9577464788732394,
 'VBZ': 1.0,
 'NNP': 1.0,
 'VBN': 0.0,
 'CC': 1.0,
 'NNS': 1.0,
 'PRP': 1.0,
 'RB': 1.0,
 'VBP': 1.0,
 'JJ': 1.0,
 'MD': 1.0,
 'EX': 1.0,
 'VB': 1.0,
 'CD': 1.0,
 'VBG': 1.0,
 'IN': 1.0,
 '.': 1.0,
 'DT': 1.0,
 'NN': 1.0,
 'JJR': 0.0}

In [26]:
import neurox.analysis.visualization as visualization

# Visualize a single sentence
visualization.visualize_activations(
    tokens["source"][9],
    activations[9][:, 2]
)

In [27]:
# Visualize entire dataset using transformer model
from IPython.display import display
from neurox.analysis.visualization import TransformersVisualizer

viz = TransformersVisualizer('bert-base-uncased')
for s_idx in range(len(tokens["source"])):
    display(viz(tokens["source"][s_idx], 0, 5))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
## Filtered visualization
for s_idx in range(len(tokens["source"])):
    display(viz(tokens["source"][s_idx], 0, 5, filter_fn="top_tokens"))

### Top words per neuron

In [29]:
from neurox.analysis.corpus import get_top_words

In [30]:
print(get_top_words(tokens, activations, 0))

[('can', 1.0), ('<<UNKNOWN>>', 0.8374996042373963), ('this', 0.7417334743790905), ('There', 0.6959218411824226), ('sentence', 0.6481815429944557), ('red', 0.6246387699270328), ('!', 0.6154971458274725), ('than', 0.6124322592376957), ('more', 0.5898240359082019), ('word', 0.5344059976535159), ('insightful', 0.528528552864625), ('be', 0.5191353847779002), ('test', 0.5111003941732798), ('This', 0.47618579649353443), ('toolkit', 0.4664674003831317), ('question', 0.465488154669146), ('you', 0.46063812941643245), ('quickly', 0.45942280447544265), ('is', 0.4592116301522393), ('10000', 0.45776577348935954), ('the', 0.4574250986964273), ('car', 0.42620446047912103), ('?', 0.3775879389218368), ('Hope', 0.3771822296748212), ('a', 0.3493998052587224), ('neuron', 0.3336644772676553), ('The', 0.3088258036671118), ('It', 0.2977828964753988), ('NeuroX', 0.2847366657793814), ('zoomed', 0.28037167742755276), ('beautiful', 0.26925516302234914), ('analysis', 0.2679976932899512), ('used', 0.261948793758052